In [6]:
import numpy as np
from gurobipy import *
import pandas as pd
np.random.seed(1)

In [2]:
def Corte_Entrada(i,A):
    Ce = []
    for (x,y) in A:
        if y== i:
            Ce = Ce + [(x,y)]
    return Ce

def Corte_Salida(i):
    Cs = []
    for (x,y) in A:
            if x==i:
                Cs = Cs + [(x,y)]
    return Cs

def Corte_Entrada_Nodo(i,A):
    Ce = []
    for (x,y) in A:
        if y== i:
            Ce = Ce + [x]
    return Ce

def Corte_Salida_Nodo(i,A):
    Cs = []
    for (x,y) in A:
            if x==i:
                Cs = Cs + [y]
    return Cs

## MODELO 1

In [1]:
def Trabajo(gap=0,time=3600):
    m = Model()

    # Crear las variables
    x = m.addVars([(k, m, i, j) for (k, m, i, j) in K], name="x", vtype=GRB.BINARY)
    y = m.addVars([(k, m, j) for k in range (1,n1+1) for m in M[k] for j in V_D], name="y", vtype=GRB.BINARY)
    a = m.addVars([(k, m, i, d) for k in range (1,n1+1) for m in M[k] for i in V for d in range(1,1+n2)] , name = "a", vtype = GRB.INTEGER, lb=0)
    z = m.addVars([(k, m) for k in range (1,n1+1) for m in M[k] ], name = "z", vtype = GRB.BINARY )
    u = m.addVars( V, name="u", vtype = GRB.INTEGER, lb=1, ub= n)
    m.update()
    
    #Función Objetivo
    t_V = {(k, m, i, j): c[i,j] for(k,m,i,j) in K}

    m.setObjective(x.prod(t_V) + z.prod(c_M) , GRB.MINIMIZE)

    #Restricciones
    
    for k,b,i,j in K:
        if type(i) == str or type(j) == str:
            continue
        m.addConstr(
            (u[j]  >= u[i] + (1 + n)*x[k,b,i,j] - n)
        )
    
    
    m.addConstrs(
        ( x.sum(k, m,"O"+str(k) ,"*")  == z[k,m] for k in range(1,n1+1) for m in M[k]) 
    )

    m.addConstrs(
        quicksum(x[k, m, j[0], "D"+str(p)]  for p in range(1,n2+1) for j in Corte_Entrada("D"+str(p), At) ) == z[k, m]  for k in range(1,n1+1) for m in M[k]
    )
    
    m.addConstrs(
        ( x.sum(k, m,"*" ,i)  == y[k, m, i]  for k in range(1,n1+1) for m in M[k] for i in V) 
    ) 
    
    m.addConstrs(
        ( x.sum(k, m,"*","D"+str(p) )  == y[k, m, "D"+str(p)] for k in range(1,n1+1) for m in M[k] for p in range(1,n2+1)) 
    )
    
    m.addConstrs(
        ( x.sum(k, m, "*",s)  == x.sum(k, m ,s,"*") for k in range(1,n1+1) for m in M[k] for s in V)
    )

#     m.addConstrs(
#         ( x.sum(k, "*","O"+str(k),"*") <= len(M[k]) for k in range(1,n1+1))
#     )
    
    m.addConstrs(
        (a.sum(k, m, "*", p) <= Dem[p]*y[k, m, "D"+str(p)] for k in range(1,n1+1) for m in M[k] for p in range(1,1+n2))
    )
    
    m.addConstrs(
        (a.sum("*", "*", i, "*") <= OF[i] for i in V )
    )


    m.addConstrs(
        (a.sum(k, m,"*", "*") <= Q[k, m]  for k in range(1,n1+1) for m in M[k])
    )

    m.addConstrs(
       (a[k, m, i, p]<= OF[i]*y[k, m, i] for p in range(1,n2+1)  for k in range(1,n1+1) for m in M[k] for i in V )
    )

    m.addConstrs(
        (a.sum("*", "*", "*", p) == Dem[p] for p in range(1,n2+1))
    )

    m.Params.MIPGap = gap
    m.Params.TimeLimit = time
    
    m.optimize()
    par={}
    clasi = {}
    producto = {}
    
    try:
    
        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
            par = {"Obj":m.objval, "GAP":abs(m.objval-m.ObjBoundC) /abs(m.objval), "Time": m.Runtime}


        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
        # Recuperar los valores de las variables
            vx = m.getAttr('x', x)
            print('Arcos seleccionados:')
            for (k,m1,i,j) in K:
                if vx[k,m1,i,j] >= 0.99:
                    print('Deposito {} usa el auto{}:= {} -> {}'.format(k,m1, i, j))
                    if ("O"+str(k)+"-"+str(m1) in clasi.keys() )== False:
                        clasi.update({"O"+str(k)+"-"+str(m1) : [(i,j)]})
                    else:
                        clasi["O"+str(k)+"-"+str(m1)]  = clasi["O"+str(k)+"-"+str(m1)] + [(i,j)]


        print("-"*20)


        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
             for k in range (1,n1+1):
                    for m1 in M[k]:
                        for i in V :
                            for d in range(1,1+n2):
                                if a[k,m1,i,d].x>0:
                                    c_prod = a[k,m1,i,d].x
                                    if (("O"+str(k),m1) in producto.keys() )== False:
                                            producto.update({("O"+str(k),m1): [(i, c_prod,d)] })
                                    else:
                                        producto[("O"+str(k),m1)] = producto[("O"+str(k),m1)] + [(i, c_prod,d)]
                                    print("El auto {} del deposito {} recoge {} del producto en {} para {}".format(m1,k,c_prod,i,d))

        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
            for k in range (1,n1+1):
                for m1 in M[k]: 
                    for j in V_D:
                        if y[k,m1,j].x>0.99:
                            print("el auto {} del deposito {} visita {}".format(m1, k, j))
    except:
        print("TIME LIMIT!")
        
        
    return clasi, producto, par

## MODELO 2

In [2]:

def Trabajo1 (gap=0, time = 3600,Autos =[]):
    np.random.seed(1)
    m = Model()
    # Crear las variables
    print("CREANDO VARIABLES")
    x = m.addVars([(k, m, i, j) for (k, m, i, j) in K], name="x", vtype=GRB.BINARY)
    y = m.addVars([(k, m, j) for k in range (1,n1+1) for m in M[k] for j in V_D], name="y", vtype=GRB.BINARY)
    a = m.addVars([(k, m, i, d) for k in range (1,n1+1) for m in M[k] for i in V for d in range(1,1+n2)] , name = "a", vtype = GRB.INTEGER, lb=0)
    u = m.addVars( V, name="u", vtype = GRB.INTEGER, lb=1, ub= n)
    m.update()
    
    #Función Objetivo
    t_V = {(k, m, i, j): c[i,j] for(k,m,i,j) in K}
    C_M ={ (k, m, "O"+str(k), i): c_M[k,m] for k in range(1,n1+1) for m in M[k] for i in V}
    
    print("CREANDO FUNCION OBJ")
    m.setObjective(x.prod(t_V) + x.prod(C_M) , GRB.MINIMIZE)

    #Restricciones
    
    print("CREANDO RESTRICCIONES")
    
    for k,b,i,j in K:
        if type(i) == str or type(j) == str:
            continue
        m.addConstr(
            (u[j]  >= u[i] + (1 + n)*x[k,b,i,j] - n)
        )
    
    m.addConstrs(
        ( x.sum(k, m,"O"+str(k) ,"*")  <= 1 for k in range(1,n1+1) for m in M[k]) 
    )

    m.addConstrs(
        quicksum(x[k, m, j[0], "D"+str(p)]  for p in range(1,n2+1) for j in Corte_Entrada("D"+str(p), At) ) <=1  for k in range(1,n1+1) for m in M[k]
    )
    
    m.addConstrs(
        ( x.sum(k, m,"*" ,i)  == y[k, m, i]  for k in range(1,n1+1) for m in M[k] for i in V) 
    ) 
    
    m.addConstrs(
        ( x.sum(k, m,"*","D"+str(p) )  == y[k, m, "D"+str(p)] for k in range(1,n1+1) for m in M[k] for p in range(1,n2+1)) 
    )
    
    m.addConstrs(
        ( x.sum(k, m, "*",s)  == x.sum(k, m ,s,"*") for k in range(1,n1+1) for m in M[k] for s in V)
    )

#     m.addConstrs(
#         ( x.sum(k, "*","O"+str(k),"*") <= len(M[k]) for k in range(1,n1+1))
#     )
    
    m.addConstrs(
        (a.sum(k, m, "*", p) <= Dem[p]*y[k, m, "D"+str(p)] for k in range(1,n1+1) for m in M[k] for p in range(1,1+n2))
    )
    
    m.addConstrs(
        (a.sum("*", "*", i, "*") <= OF[i] for i in V )
    )


    m.addConstrs(
        (a.sum(k, m,"*", "*") <= Q[k, m]  for k in range(1,n1+1) for m in M[k])
    )

    m.addConstrs(
        (a[k, m, i, p]<= OF[i]*y[k, m, i] for p in range(1,n2+1)  for k in range(1,n1+1) for m in M[k] for i in V )
    )

    m.addConstrs(
        (a.sum("*", "*", "*", p) == Dem[p] for p in range(1,n2+1))
    )

    m.Params.MIPGap = gap
    m.Params.TimeLimit= time
    print("OPTIMIZE")
    
    
    if len(Autos)==0:
        m.optimize()
    else:
        
        print("ENTREEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE!")
        for auto in Autos:
            #sum( [x["O"+str(auto[0]), auto[1], "O"+str(auto[0]), i].Start for i in Corte_Salida_Nodo("O"+str(auto[0]), A) ] ) = 1
            #x[].Start  = 1
            y[auto[0], auto[1], auto[2]].Start  = 1
            
        m.optimize()
    
    
    
    
    
    
    print("-"*20)

    clasi = {}
    par = {}
    producto = {}
    print("GUARDANDO SOLUCIONES")
    
    try:
        
        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
            par = {"Obj":m.objval, "GAP":abs(m.objval-m.ObjBoundC) /abs(m.objval), "Time": m.Runtime}
        # Recuperar los valores de las variables
            vx = m.getAttr('x', x)
            print('Arcos seleccionados:')
            for (k,m1,i,j) in K:
                if vx[k,m1,i,j] >= 0.99:
                    print('Deposito {} usa el auto{}:= {} -> {}'.format(k,m1, i, j))
                    if ("O"+str(k)+"-"+str(m1) in clasi.keys() )== False:
                        clasi.update({"O"+str(k)+"-"+str(m1) : [(i,j)]})
                    else:
                        clasi["O"+str(k)+"-"+str(m1)]  = clasi["O"+str(k)+"-"+str(m1)] + [(i,j)]


        print("-"*20)

        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
             for k in range (1,n1+1):
                    for m1 in M[k]:
                        for i in V :
                            for d in range(1,1+n2):
                                if a[k,m1,i,d].x>0:
                                    c_prod = a[k,m1,i,d].x
                                    if (("O"+str(k),m1) in producto.keys() )== False:
                                            producto.update({("O"+str(k),m1): [(i, c_prod,d)] })
                                    else:
                                        producto[("O"+str(k),m1)] = producto[("O"+str(k),m1)] + [(i, c_prod,d)]
                                    print("El auto {} del deposito {} recoge {} del producto en {} para {}".format(m1,k,c_prod,i,d))

        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
            for k in range (1,n1+1):
                for m1 in M[k]: 
                    for j in V_D:
                        if y[k,m1,j].x>0.99:
                            print("el auto {} del deposito {} visita {}".format(m1, k, j))
    except:
        print("TIME LIMIT!")
    
    return clasi, producto, par

## LECTURA Y RESOLUCIÓN DE LAS 52 INTANCIAS 

In [7]:

from IPython.display import Markdown


df_instancias = pd.read_excel("InstanciasFinal.xlsx",sheet_name= "Sheet1", header=0, index_col=[0,1,2])


Instancias = {5: {11: [17], 14: [17], 17: [2], 18: [6]},
 10: {11: [10], 15: [4], 17: [9], 21: [11]},
 15: {6: [5], 7: [6], 18: [7], 22: [9]},
 20: {12: [2], 13: [8], 19: [10], 24: [4]},
 25: {11: [12], 16: [9], 17: [17], 24: [17]},
 30: {10: [2], 15: [3], 20: [12], 24: [16]},
 35: {6: [15], 15: [2], 19: [5], 23: [19]},
 40: {6: [4], 7: [8], 13: [8], 15: [15]},
 45: {14: [13], 19: [9], 23: [14], 25: [15]},
 50: {13: [13], 14: [14], 16: [13], 21: [10]},
 55: {9: [6], 12: [9], 23: [9], 24: [15]},
 60: {10: [6], 17: [20], 18: [2], 23: [18]},
 65: {15: [9], 19: [12], 22: [15], 25: [19]}}
    

for j in df_instancias.columns:
    s = df_instancias[j]
    for m in list(Instancias.keys()):
        for i in list(Instancias[m].keys()):
            for k in Instancias[m][i]:
                s[m,i,k] = list([int(s[m,i,k].split(",")[0][1:]), int(s[m,i,k].split(",")[1][:-1])+1]) #mas 1 por como se generan los #aleatorios


df_primeras = df_instancias

prob = 0.5

DatosInstancias3 = {}
for n in list(Instancias.keys()):
    V = tuplelist(range(1,n+1))
    np.random.seed(1)
    a1 = {(i,j): np.random.randint(35,151) for i in V for j in V if (i!=j and np.random.random()<prob)}

    for i,j in list(a1.keys()):
        if (j,i) in list(a1.keys()) and i<j:
            a1[j,i] = a1[i,j]

    a = {**a1}
    A = list(a.keys())
    for n1 in list(Instancias[n].keys()):
        O = ["O"+str(i) for i in range(1,n1+1)]
        np.random.seed(1)
        Ao = [ (i,j) for i in O for j in V if np.random.random()<0.85 ]
        np.random.seed(1)
        b1 = {(i,j): np.random.randint(35,150) for (i,j) in Ao }
        V_O = [i for i in range(1,n+1)]+O
        
        for n2 in Instancias[n][n1]:
            display (Markdown('<span style="color: #ff0000">{},{} y {}</span> '.format(n,n1,n2)))
            np.random.seed(1)
            #Creación de D
            D = ["D"+str(i) for i in range(1,n2+1)]
            np.random.seed(1)
            Ad = [ (i,j) for i in V for j in D if np.random.random()<0.85 ]
            np.random.seed(1)
            b2 = {(i,j): np.random.randint(35,150)  for (i,j) in Ad }
            V_D = [i for i in range(1,n+1)]+D

            At,c= multidict({**a, **b1, **b2})
            #Vehículos
            np.random.seed(1)
            M = {k: [i for i in range(1,
                                                  np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][0]+1,
                                                                    df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][1]+1))] for k in range(1,n1+1) }
            np.random.seed(1)
            c_M = { (k, m):  np.random.randint(350,500) for k in range(1,n1+1) for m in M[k]}
            
            
            
            
            #Creación K
            K = []
            for k in range(1,n1+1):
                for m in M[k]:
                    K = K + [(k, m, j[0], j[1]) for j in Ao if "O"+str(k) == j[0]  ]
                    K = K + [(k,m,i,j) for (i,j) in A+Ad]
                    
            #Capacidades de los vehículos
            np.random.seed(1)
            Q = {(k, m) : np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][0], df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][1]) for k in range(1,n1+1) for m in M[k]}

            #Oferta
            np.random.seed(1)
            OF = {i: np.random.randint(df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][0], df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][1]) for i in V}

            #Demanda
            np.random.seed(1)
            Dem = {p :np.random.randint(df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][0], df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][1]) for p in range(1,n2+1) }

            print(len(At))        
            print("Modelo1")
            Ruta1, Pro1, Par1 = Trabajo()
            
            print("Modelo2")
            Ruta2, Pro2, Par2 = Trabajo1()
            
            DatosInstancias3.update({(n,n1,n2): [c,M,c_M, Q, OF, Dem, Ruta1, Pro1, Par1, Ruta2, Pro2, Par2] })


<span style="color: #ff0000">5,11 y 17</span> 

132


<span style="color: #ff0000">5,14 y 17</span> 

146


<span style="color: #ff0000">5,17 y 2</span> 

...

3759


<span style="color: #ff0000">65,22 y 15</span> 

4088


<span style="color: #ff0000">65,25 y 19</span> 

4465


## IMPRIMIR INDICADORES PARA COMPARAR MODELOS

In [14]:
for i in DatosInstancias3.keys():
    print("____"*20)
    print("INSTANCIA {}".format(i))
    print("Para el modelo 1")
    print(DatosInstancias3[i][-4])
    print("Para el modelo 2")
    print(DatosInstancias3[i][-1] )

________________________________________________________________________________
INSTANCIA (5, 11, 17)
Para el modelo 1
{'Obj': 18252.0, 'GAP': 0.030950213438445572, 'Time': 3600.240955352783}
Para el modelo 2
{'Obj': 18252.0, 'GAP': 0.10923125833652335, 'Time': 3600.107135772705}
________________________________________________________________________________
INSTANCIA (5, 14, 17)
Para el modelo 1
{'Obj': 18053.0, 'GAP': 0.16913913994777016, 'Time': 3600.1114711761475}
Para el modelo 2
{'Obj': 18053.0, 'GAP': 0.12014844632187625, 'Time': 3600.164560317993}
________________________________________________________________________________
INSTANCIA (5, 17, 2)
Para el modelo 1
{'Obj': 6903.0, 'GAP': 0.0, 'Time': 2131.6100425720215}
Para el modelo 2
{'Obj': 6903.0, 'GAP': 0.0, 'Time': 218.75473976135254}
________________________________________________________________________________
INSTANCIA (5, 18, 6)
Para el modelo 1
{'Obj': 9480.0, 'GAP': 0.02954470565455325, 'Time': 3600.094406127929

## GUARDAR LAS SOLUCIONES EN *.txt

In [48]:
# open file for writing
f = open("SOLUCIONES_MODELO1y2.txt","w")

# write file
f.write( str(DatosInstancias3) )

# close file
f.close()

## COLOCACIÓN DE INDICADORES EN CADA INSTANCIA

In [34]:
df_instancias = pd.read_excel("InstanciasFinal.xlsx",sheet_name= "Sheet1", header=0, index_col=[0,1,2])
dfcolumns = [("Parametros",i) for i in df_instancias.columns]
df_instancias.columns = dfcolumns


a = pd.Series([DatosInstancias3[i][-4]["Obj"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 1","OBJ")] = a

a = pd.Series([DatosInstancias3[i][-4]["GAP"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 1","GAP")] = a

a = pd.Series([DatosInstancias3[i][-4]["Time"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 1","Tiempo")] = a

a = pd.Series([DatosInstancias3[i][-1]["Obj"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 2","OBJ")] = a

a = pd.Series([DatosInstancias3[i][-1]["GAP"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 2","GAP")] = a

a = pd.Series([DatosInstancias3[i][-1]["Time"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 2","Tiempo")] = a


df_instancias.head()

(Parametros, Rango de \nVehículo para cada deposito)  \
5  11 17                                             [2, 5]     
   14 17                                             [2, 3]     
   17 2                                              [1, 4]     
   18 6                                              [1, 3]     
10 11 10                                             [1, 2]     

         (Parametros, Rango de \ncapacidad)  \
5  11 17                         [119, 122]   
   14 17                           [98, 99]   
   17 2                            [19, 22]   
   18 6                            [44, 46]   
10 11 10                         [120, 124]   

         (Parametros, Oferta para\n cada lugar de abastecimiento)  \
5  11 17                                         [525, 527]         
   14 17                                         [545, 547]         
   17 2                                            [64, 67]         
   18 6                                          [157, 161]         
10 11 10                                         [133, 137]         

         (Parametros, Rango Demanda)  (MODELO 1, OBJ)  (MODELO 1, GAP)  \
5  11 17                  [129, 154]          18252.0         0.030950   
   14 17                  [121, 160]          18053.0         0.169139   
   17 2                   [130, 158]           6903.0         0.000000   
   18 6                   [124, 130]           9480.0         0.029545   
10 11 10                  [121, 132]           9094.0         0.000000   

          (MODELO 1, Tiempo)  (MODELO 2, OBJ)  (MODELO 2, GAP)  \
5  11 17         3600.240955          18252.0         0.109231   
   14 17         3600.111471          18053.0         0.120148   
   17 2          2131.610043           6903.0         0.000000   
   18 6          3600.094406           9478.0         0.028174   
10 11 10         2042.168102           9094.0         0.000000   

          (MODELO 2, Tiempo)  
5  11 17         3600.107136  
   14 17         3600.164560  
   17 2           218.754740  
   18 6          3600.111372  
10 11 10          237.849745

In [35]:
df_instancias.columns = pd.MultiIndex.from_tuples(df_instancias)

In [36]:
df_instancias.to_excel("RESULTADOS_INSTANCIAS_FINAL.xlsx")

## GUARDAR RESULTADOS SOLO MODELO 2  

In [47]:
df_instancias = pd.read_excel("InstanciasFinal.xlsx",sheet_name= "Sheet1", header=0, index_col=[0,1,2])
dfcolumns = [("Parametros",i) for i in df_instancias.columns]
df_instancias.columns = dfcolumns

a = pd.Series([DatosInstancias3[i][-1]["Obj"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 2","OBJ")] = a

a = pd.Series([DatosInstancias3[i][-1]["GAP"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 2","GAP")] = a

a = pd.Series([DatosInstancias3[i][-1]["Time"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 2","Tiempo")] = a

df_instancias.columns = pd.MultiIndex.from_tuples(df_instancias)
df_instancias.to_excel("MODELO2_INSTANCIAS_FINAL.xlsx")